In [5]:
import pennylane as qml
from pennylane import numpy as np

In [7]:
U11 = [0.58939515 - 0.13274642j, 0.64381996 + 0.46956108j, 0.79653634 + 0.02283875j, -0.38478345 - 0.46577886j]
U12 = [-0.25432824 + 0.63921011j, -0.62853537 - 0.3628648j, 0.56165482 + 0.45964273j, 0.50800259 - 0.46390278j]
U13 = [0.18203758 + 0.17691194j, -0.73088329 - 0.63354092j, 0.96399606 - 0.07922173j, 0.25139533 - 0.03515543j]

In [8]:
U11 = np.array(U11).reshape((2, 2))
U12 = np.array(U12).reshape((2, 2))
U13 = np.array(U13).reshape((2, 2))

In [9]:
U21 = [-0.38244183 - 0.12961519j, -0.58523477 - 0.07874502j, 0.30743697 - 0.1900339j,
       0.35244239 - 0.48310622j, 0.0520515 + 0.11832368j, -0.31650125 - 0.29741055j,
       -0.43985789 + 0.65802989j, -0.22420903 - 0.3433891j, 0.80468183 - 0.37828889j,
       -0.17001017 - 0.01590874j, -0.00860685 - 0.28232261j, -0.10595894 - 0.29869833j,
       0.17166715 + 0.01197327j, -0.3397648 + 0.56398292j, -0.37158034 + 0.15823791j,
       0.54409558 + 0.27878125j]
U22 = [0.39021299 + 0.19550829j, 0.51227196 + 0.31891725j, 0.29405574 - 0.23122613j,
       0.05455699 + 0.54997163j, -0.01534597 + 0.4880493j, 0.32273224 - 0.0277232j,
       -0.78572909 - 0.00924454j, -0.19764638 - 0.01131186j, 0.61343005 + 0.33688749j,
       -0.55122531 + 0.10760665j, -0.08187687 - 0.30746768j, 0.22104702 - 0.21137614j,
       0.1761403 + 0.22420457j, 0.45391289 - 0.09736075j, 0.37273568 + 0.04937567j,
       -0.14723682 - 0.73494378j]
U23 = [0.09993947 + 0.22310773j, -0.52519719 - 0.19872825j, 0.49386529 + 0.09293278j,
       -0.45451301 + 0.40717139j, 0.15789114 + 0.17890024j, 0.01504621 - 0.04207803j,
       -0.6909489 + 0.35722228j, -0.03586294 + 0.5785878j, 0.40172155 - 0.35194883j,
       -0.31727546 - 0.66402832j, -0.20322529 - 0.28188923j, 0.2074951 - 0.0966345j,
       -0.75580237 + 0.16375204j, -0.07412908 - 0.36821951j, 0.02474916 - 0.14514768j,
       0.40442414 + 0.27500556j]

In [10]:
U21 = np.array(U21).reshape((4, 4))
U22 = np.array(U22).reshape((4, 4))
U23 = np.array(U23).reshape((4, 4))

In [86]:
import pennylane as qml
from pennylane import numpy as np


def qpe_qnode(U, eigvec, n_estimation_wires):

    n_trg_wires = int(np.log2(len(U)))
    target_wires = np.arange(0, n_trg_wires).tolist()
    estimation_wires = np.arange(n_trg_wires, n_estimation_wires+n_trg_wires).tolist()

    dev = qml.device("default.qubit", wires=estimation_wires+target_wires)

    @qml.qnode(dev)
    def qnode():
        qml.MottonenStatePreparation(eigvec, wires=target_wires)
        qml.QuantumPhaseEstimation(
            U,
            target_wires=target_wires,
            estimation_wires=estimation_wires
        )
        return qml.probs(wires=estimation_wires)
    return qnode

def estimate_phases(U, starting_state):
    """Use QPE to estimate all eigenvalues of a unitary matrix when
    the eigenvectors are unknown.

    Your code will be tested on 1-qubit unitaries, and 2-qubit unitaries.

    Use the QuantumPhaseEstimation template; and implement as many helper
    functions as needed.

    Args:
        U (array[complex]): A random 1-qubit or 2-qubit unitary matrix.
        starting_state (array[complex]): The state that the target register
            starts in; it is guaranteed to be in a non-trivial linear combination
            of the eigenstates of U.

    Returns:
        QNode, List[complex]: The QNode, and a list of complex numbers that are
        estimates of the phases of U. These numbers should be the complex values
        exp[2*pi*i*theta], where theta is the real-valued phase estimate
        obtained from the QPE procedure. Do not worry about the order; the test
        function will sort the values.
    """
    # Should be length 2 or 4, depending on size of input
    eigenvalues = []

    # Note: using 12 estimation wires should yield results that pass the test
    # cases at the specified tolerance. But if it is too much for your hardware,
    # try reducing to 11; the tolerance in the test files is fairly generous.
    n_estimation_wires = 12

    # YOUR CODE HERE
    # Set up a QNode that prepares the target register and performs QPE.
    # Then, run it and use the results to determine the eigenvalues.
    def run_qpe():
        qnode = qpe_qnode(U, starting_state, n_estimation_wires)
        return qnode

    # eigvals, eigvecs = np.linalg.eig(U)

    num_e_vals = U.shape[0]

    qnode = run_qpe()

    measurements = qnode()

    eig_vals = np.argsort(measurements)[-num_e_vals:]
    eig_vals = eig_vals/(2**12)
    eig_vals = np.exp(1j*2*np.pi*eig_vals)

    return qnode, eig_vals

In [90]:
eigenvector = np.array([ 0.878242  +0.j        , -0.45675296-0.14166061j])
node, e_vector = estimate_phases(U11, eigenvector)

In [91]:
e_vector

tensor([-0.79490713-0.60673113j,  0.99997059+0.00766983j], requires_grad=True)

In [17]:
result = node()

In [19]:
result

tensor([1.98510234e-05, 3.11091178e-05, 5.56180221e-05, ...,
        7.73724139e-06, 1.01060410e-05, 1.37643186e-05], requires_grad=True)

In [58]:
e_vector

array([ 0.99997102+0.00761294j, -0.79535932-0.60613822j])

In [59]:
e_vector

array([ 0.99997102+0.00761294j, -0.79535932-0.60613822j])

In [75]:
e_vals = np.argsort(result)[-2:]
e_vals

tensor([2473,    5], requires_grad=True)

In [76]:
result[np.argsort(result)[-2:]]

tensor([0.27546815, 0.35474221], requires_grad=True)

In [77]:
e_vals = e_vals/(2**12)
e_vals

tensor([0.60375977, 0.0012207 ], requires_grad=True)

In [78]:
np.exp(1j*2*np.pi*e_vals)

tensor([-0.79490713-0.60673113j,  0.99997059+0.00766983j], requires_grad=True)

In [23]:
U11.shape[0]

2

In [39]:
np.argsort([12,0,2,8])

tensor([1, 2, 3, 0], requires_grad=True)

In [54]:
np.argsort([12,0,2,8])[-2:]

tensor([3, 0], requires_grad=True)